# OpenAI Embedding API 활용 가이드

## 1. 환경 설정

In [1]:
!pip install openai numpy scikit-learn tenacity

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## 2. API 키 설정

In [ ]:
# import os
# os.environ['OPENAI_API_KEY'] = 'your-api-key'

## 3. OpenAI Embedding API 사용하기

In [2]:
from openai import OpenAI
client = OpenAI()

In [3]:
response = client.embeddings.create(
    input="안녕",
    model="text-embedding-ada-002"
)

print(response.data[0].embedding)

[-0.013785294257104397, -0.010510955937206745, 0.008625730872154236, -0.01665612682700157, -0.012442484498023987, 0.020916074514389038, -0.0219876766204834, 0.00310565996915102, -0.009393050335347652, -0.0009335170616395772, 0.01272030733525753, 0.001994369551539421, -0.023098966106772423, -0.012607854790985584, 0.00524224853143096, 0.003004783997312188, 0.03421187028288841, 0.00012547495134640485, 0.0213923417031765, -0.021828919649124146, -0.0004977655480615795, -0.018918396905064583, -0.005119874142110348, -0.00860588625073433, 0.006284083239734173, 0.015438999980688095, 0.011695009656250477, -0.017357299104332924, 0.010973993688821793, -0.02394566498696804, 0.016113711521029472, 0.008744797669351101, -0.027623506262898445, -0.015200866386294365, 0.002733576111495495, -0.01702655851840973, -0.00310565996915102, -0.01086815632879734, 0.005999645683914423, -0.004203720949590206, 0.015359622426331043, -0.01935497671365738, 0.004084654152393341, -0.008493434637784958, -0.004799054935574

In [4]:
len(response.data[0].embedding)

1536

## 4. OpenAI Embedding API 호출 함수 만들기

In [5]:
import numpy as np
from tenacity import retry, wait_random_exponential, stop_after_attempt

In [6]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def encode(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-ada-002")
    return np.array(response.data[0].embedding)

In [7]:
emb = encode("안녕")

In [8]:
emb

array([-0.01378529, -0.01051096,  0.00862573, ..., -0.02102191,
        0.00657514,  0.00506696])

In [9]:
emb.shape

(1536,)

## 5. 예제 - 유사 질의 찾기

In [10]:
from sklearn.metrics import pairwise_distances
from pprint import pprint

In [11]:
candidates = ["오늘 날씨 알려줘", "오늘 날씨 어때", "오늘 날씨 알고싶어",
              "내일 날씨 알려줘", "내일 날씨 어때", "내일 날씨 알고싶어",
              "아이유 노래 틀어", "아이유 노래 실행해",
              "7시 알람",
              "유재석이 누구야", "신동엽이 누구야",
              "tv 꺼줘", "tv 켜줘", "티비 꺼줘", "티비 켜줘",
              "그만", "멈춰", "꺼",
              "코로나 확진자 몇명이야",
              "코로나 알려줘",
              "코로나 요즘 어때",
              "아니모르겠어",
              "오늘 코로나 십구 부여 은 환자은 수 어때",
              "코로나 확진자 알려줘",
              "웰시 코기에 대해서 알려줘",
              "오늘 코로나 바이러스 인제군 상황이 보여줘",
              "pytorch 설치방법",
              "파이토치 설치 방법",
              "텐서플로우 설치 방법",
              "텐서플로우 사용법",
              ]

In [12]:
# num candidates x dim
candidates_embs = np.stack([encode(q) for q in candidates], axis=0)

In [13]:
candidates_embs.shape

(30, 1536)

In [14]:
query = "날씨"
query_emb = encode(query)

In [15]:
query_emb

array([ 0.00687834, -0.02220354,  0.0076176 , ..., -0.02095644,
        0.00286223,  0.00783616])

In [16]:
query_emb.reshape(1, -1).shape

(1, 1536)

In [17]:
dist = pairwise_distances(query_emb.reshape(1,-1), candidates_embs, metric="cosine").flatten()

In [18]:
dist

array([0.11105453, 0.11274948, 0.1119487 , 0.08653174, 0.08084257,
       0.08839923, 0.22191962, 0.22214461, 0.18679325, 0.23662231,
       0.20756623, 0.20176745, 0.21517844, 0.22489222, 0.23202675,
       0.22751676, 0.19993512, 0.19736515, 0.21797959, 0.17793381,
       0.18546418, 0.20134552, 0.23335702, 0.19461098, 0.21329587,
       0.20131896, 0.26369178, 0.25542084, 0.2428666 , 0.2325162 ])

In [19]:
ordered_by_distance_idx = dist.argsort()

In [20]:
ordered_by_distance_idx

array([ 4,  3,  5,  0,  2,  1, 19, 20,  8, 23, 17, 16, 25, 21, 11, 10, 24,
       12, 18,  6,  7, 13, 15, 14, 29, 22,  9, 28, 27, 26])

In [21]:
target_ordered_by_distance = [candidates[idx] for idx in ordered_by_distance_idx]
dist_ordered_by_distance = [dist[idx] for idx in ordered_by_distance_idx]

In [22]:
target_ordered_by_distance

['내일 날씨 어때',
 '내일 날씨 알려줘',
 '내일 날씨 알고싶어',
 '오늘 날씨 알려줘',
 '오늘 날씨 알고싶어',
 '오늘 날씨 어때',
 '코로나 알려줘',
 '코로나 요즘 어때',
 '7시 알람',
 '코로나 확진자 알려줘',
 '꺼',
 '멈춰',
 '오늘 코로나 바이러스 인제군 상황이 보여줘',
 '아니모르겠어',
 'tv 꺼줘',
 '신동엽이 누구야',
 '웰시 코기에 대해서 알려줘',
 'tv 켜줘',
 '코로나 확진자 몇명이야',
 '아이유 노래 틀어',
 '아이유 노래 실행해',
 '티비 꺼줘',
 '그만',
 '티비 켜줘',
 '텐서플로우 사용법',
 '오늘 코로나 십구 부여 은 환자은 수 어때',
 '유재석이 누구야',
 '텐서플로우 설치 방법',
 '파이토치 설치 방법',
 'pytorch 설치방법']

In [23]:
dist_ordered_by_distance

[0.08084256645323729,
 0.08653174485249282,
 0.08839923315496123,
 0.11105453089340611,
 0.11194870177910454,
 0.11274948197226564,
 0.17793380958018346,
 0.185464181338293,
 0.18679325290426352,
 0.1946109809289609,
 0.19736514539928618,
 0.19993512168385408,
 0.20131895679886957,
 0.20134551798243894,
 0.2017674495640509,
 0.20756623290513532,
 0.21329587054919275,
 0.21517844279253107,
 0.21797958701017706,
 0.22191961896286394,
 0.22214460503439315,
 0.22489222096882877,
 0.22751675898846457,
 0.23202675408232776,
 0.2325162031354231,
 0.2333570235820246,
 0.23662230735193868,
 0.24286659826566814,
 0.25542084229691764,
 0.2636917790620994]

In [24]:
# Print
print(f"query: {query}")
print("ordered_by_distance")
pprint([(t, d) for t, d in zip(target_ordered_by_distance, dist_ordered_by_distance)])

query: 날씨
ordered_by_distance
[('내일 날씨 어때', 0.08084256645323729),
 ('내일 날씨 알려줘', 0.08653174485249282),
 ('내일 날씨 알고싶어', 0.08839923315496123),
 ('오늘 날씨 알려줘', 0.11105453089340611),
 ('오늘 날씨 알고싶어', 0.11194870177910454),
 ('오늘 날씨 어때', 0.11274948197226564),
 ('코로나 알려줘', 0.17793380958018346),
 ('코로나 요즘 어때', 0.185464181338293),
 ('7시 알람', 0.18679325290426352),
 ('코로나 확진자 알려줘', 0.1946109809289609),
 ('꺼', 0.19736514539928618),
 ('멈춰', 0.19993512168385408),
 ('오늘 코로나 바이러스 인제군 상황이 보여줘', 0.20131895679886957),
 ('아니모르겠어', 0.20134551798243894),
 ('tv 꺼줘', 0.2017674495640509),
 ('신동엽이 누구야', 0.20756623290513532),
 ('웰시 코기에 대해서 알려줘', 0.21329587054919275),
 ('tv 켜줘', 0.21517844279253107),
 ('코로나 확진자 몇명이야', 0.21797958701017706),
 ('아이유 노래 틀어', 0.22191961896286394),
 ('아이유 노래 실행해', 0.22214460503439315),
 ('티비 꺼줘', 0.22489222096882877),
 ('그만', 0.22751675898846457),
 ('티비 켜줘', 0.23202675408232776),
 ('텐서플로우 사용법', 0.2325162031354231),
 ('오늘 코로나 십구 부여 은 환자은 수 어때', 0.2333570235820246),
 ('유재석이 누구야', 0.2366223

문서 분류, 유사도 계산 등 다양한 문제에 임베딩을 활용하는 방법을 소개합니다.

API 호출을 효율적으로 관리하고, 임베딩을 효과적으로 사용하는 다양한 팁을 제공합니다.

잠재적인 오류를 처리하고 문제를 디버깅하는 방법에 대해 설명합니다.

Embedding API와 관련된 추가 자료와 학습 리소스를 제공합니다.